# Assignment 08
## Addison Flack
### Deep Learning

---

### Imports:


In [2]:
import numpy as np
import arcpy
from arcgis.gis import GIS
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [3]:
gis = GIS()
arcpy.env.overwriteOutput = True

### Exporting Training Data for Deep Learning (Image Chips)

In [4]:
with arcpy.EnvManager(cellSize="5b1b6fb2-5024-4681-a175-9b667174f48c.tif"):
    arcpy.ia.ExportTrainingDataForDeepLearning(
        in_raster="5b1b6fb2-5024-4681-a175-9b667174f48c.tif",
        out_folder=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\imagechips",
        in_class_data="PalmTraining",
        image_chip_format="JPEG",
        tile_size_x=448,
        tile_size_y=448,
        stride_x=128,
        stride_y=128,
        output_nofeature_tiles="ONLY_TILES_WITH_FEATURES",
        metadata_format="PASCAL_VOC_rectangles",
        start_index=0,
        class_value_field="Classvalue",
        buffer_radius=0,
        in_mask_polygons=None,
        rotation_angle=0,
        reference_system="MAP_SPACE",
        processing_mode="PROCESS_AS_MOSAICKED_IMAGE",
        blacken_around_feature="NO_BLACKEN",
        crop_mode="FIXED_SIZE",
        in_raster2=None,
        in_instance_data=None,
        instance_class_value_field=None,
        min_polygon_overlap_ratio=0
    )

### Training the Deep Learning Model

In [1]:
with arcpy.EnvManager(processorType="GPU"):
    arcpy.ia.TrainDeepLearningModel(
        in_folder=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\imagechips",
        out_folder=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\classify_palms",
        max_epochs=50,
        model_type="SSD",
        batch_size=8,
        arguments="grids #;zooms 1.0;ratios '[1.0, 1.0]';chip_size 224;resize_to #;monitor valid_loss",
        learning_rate=None,
        backbone_model="RESNET34",
        pretrained_model=None,
        validation_percentage=10,
        stop_training="CONTINUE_TRAINING",
        freeze="FREEZE_MODEL"
    )


epoch     train_loss  valid_loss  average_precision  time    
0         51.267014   56.545658   0.325367           02:07     
1         47.485737   54.776711   0.351344           02:01     
2         45.387470   54.390320   0.403974           02:01     
3         46.863964   53.289280   0.386566           01:59     
4         44.040192   51.733196   0.433169           02:01     
5         41.142342   51.266590   0.379536           02:01     
6         43.121300   50.169334   0.456854           01:59     
7         38.991478   48.976582   0.426959           02:00     
8         39.569672   46.887211   0.453131           01:59     
9         39.804722   46.776257   0.457265           01:59     
10        39.098370   44.551880   0.527326           02:00     
11        38.265942   44.331921   0.520812           02:00     
12        36.971897   43.981525   0.524389           01:59     
13        36.011009   43.851364   0.496725           02:00     
14        35.398724   43.845001   0.510039

### Detect Objects Using Deep Learning

##### The first of these is just a test of the parameters on a smaller processing extent, the 'detection area' bookmarked area. Once the parameters are set properly and detection seems to be working well, the tool will be run over the full image.

In [1]:
with arcpy.EnvManager(extent='-19519373.1570595 -2403692.99550326 -19519228.2616025 -2403572.24928915 PROJCS["unnamed_ellipse_Mercator_2SP",GEOGCS["GCS_unnamed_ellipse",DATUM["D_unknown",SPHEROID["Unknown",6378137.0,0.0]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_2SP"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",0.0],PARAMETER["standard_parallel_1",0.0],UNIT["Meter",1.0]]', cellSize="5b1b6fb2-5024-4681-a175-9b667174f48c.tif", processorType="GPU"):
    out_classified_raster = arcpy.ia.DetectObjectsUsingDeepLearning(
        in_raster="5b1b6fb2-5024-4681-a175-9b667174f48c.tif",
        out_detected_objects=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\Kolovai.gdb\DetectedPalms",
        in_model_definition=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\classify_palms\classify_palms.dlpk",
        arguments="threshold 0.2;nms_overlap 0.1;batch_size 8",
        run_nms="NO_NMS",
        confidence_score_field="Confidence",
        class_value_field="Class",
        max_overlap_ratio=0,
        processing_mode="PROCESS_AS_MOSAICKED_IMAGE"
    )
out_classified_raster.save(None)

AttributeError: 'ArcGISObjectDetector' object has no attribute 'updatePixels'


AttributeError: ResultObject: Get attribute save not found

The error above is small, and for a single line of code - the tool still ran properly, and the output worked as desired.

### Non Maximum Suppression

The purpose of this tool is to change the overlap parameters in order to better map and represent the palm trees wtih the deep learning object detection.

In [2]:
arcpy.ia.NonMaximumSuppression(
    in_featureclass="DetectedPalms",
    confidence_score_field="Confidence",
    out_featureclass=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\Kolovai.gdb\DetectedPalms_NMS",
    class_value_field="Class",
    max_overlap_ratio=0.5
)

<geoprocessing server result object object at 0x0000024211ACCE10>

In [ ]:
from arcgis.gis import GIS
from arcgis.features import SpatialDataFrame
gis = GIS(url, user, password)
sdf = SpatialDataFrame.from_featureclass(r"")
map1 = gis.map() #local region can be added in parentheses
map1
sdf.plot(kind='map', map_widget= map1)‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍

## Estimating Vegetation Health

### Calculating VARI and Extracting VARI to Coconut Palms

The first step in this process is calcualting the VARI - however, this is done with the 'ratser functions', and is not a geoprocessing tool. After I calculated the VARI, we can use it to create points of coconut palms.

### Feature to Point - Palm Tree Points

In [3]:
arcpy.management.FeatureToPoint(
    in_features="Detected Palm Trees",
    out_feature_class=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\Kolovai.gdb\PalmTree_Points",
    point_location="CENTROID"
)

<Result 'D:\\Users\\aflack\\Prog_for_GIS_Assignment08\\Palm_Tree_Detection\\Kolovai\\Kolovai.gdb\\PalmTree_Points'>

### Buffering the Palm Tree Points

In [4]:
arcpy.analysis.PairwiseBuffer(
    in_features="PalmTree_Points",
    out_feature_class=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\Kolovai.gdb\PalmTree_Buffer",
    buffer_distance_or_field="3 Meters",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR",
    max_deviation="0 Meters"
)

<Result 'D:\\Users\\aflack\\Prog_for_GIS_Assignment08\\Palm_Tree_Detection\\Kolovai\\Kolovai.gdb\\PalmTree_Buffer'>

### Zonal Statistics as Table

In this scenario, Zonal Statistics as Table is summarizing the VARI raster (showing vegetation health) throughout the buffers.

In [5]:
arcpy.ia.ZonalStatisticsAsTable(
    in_zone_data="PalmTree_Buffer",
    zone_field="ORIG_FID",
    in_value_raster="VARI",
    out_table=r"D:\Users\aflack\Prog_for_GIS_Assignment08\Palm_Tree_Detection\Kolovai\Kolovai.gdb\MeanVARI_per_Palm",
    ignore_nodata="DATA",
    statistics_type="MEAN",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=90,
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

<geoprocessing server result object object at 0x0000024226399690>

### Joining the Field

Then, we can simply use 'Join Field' to get the data from the Zonal Statistics as Table back into the Buffer layer.

In [6]:
arcpy.management.JoinField(
    in_data="PalmTree_Buffer",
    in_field="ORIG_FID",
    join_table="MeanVARI_per_Palm",
    join_field="ORIG_FID",
    fields="MEAN",
    fm_option="NOT_USE_FM",
    field_mapping=None
)

<Result 'PalmTree_Buffer'>

#### From here, the data can just be symbolized under the symbology tab.